In [1]:
#import sys
#sys.path.append("/Users/franciumpnc/Documents/ML/AQM/fastai/courses/dl1/fastai") # go to parent dir
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
#from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [3]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [4]:
PATH = "data/grocery-sales/"

In [5]:
!ls {PATH}

Grocery_test.csv      oil.csv               train.csv
Grocery_train.csv     sample_submission.csv transactions.csv
holidays_events.csv   stores.csv
items.csv             test.csv


In [6]:
!wc -l {PATH}train.csv

 125497041 data/grocery-sales/train.csv


In [7]:
!head {PATH}train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,
2,2013-01-01,25,105575,2.0,
3,2013-01-01,25,108079,1.0,
4,2013-01-01,25,108701,1.0,
5,2013-01-01,25,108786,3.0,
6,2013-01-01,25,108797,1.0,
7,2013-01-01,25,108952,1.0,
8,2013-01-01,25,111397,13.0,


In [8]:
#??pd.merge

In [9]:
table_names = ['train', 'stores', 'items', 'transactions', 'oil', 'holidays_events', 'test']
for i in table_names:
    print(i)

train
stores
items
transactions
oil
holidays_events
test


In [10]:
df_all=pd.read_csv(f'{PATH}/train.csv',nrows=4)

In [11]:
df_all.dtypes

id               int64
date            object
store_nbr        int64
item_nbr         int64
unit_sales     float64
onpromotion    float64
dtype: object

In [12]:
df_items=pd.read_csv(f'{PATH}/items.csv',nrows=4)
df_items.dtypes

item_nbr       int64
family        object
class          int64
perishable     int64
dtype: object

In [29]:
train_types={'id':'int64','store_nbr':'int8','item_nbr':'int64','unit_sales ':'float32',
      'onpromotion':'object'}

In [14]:
%time df_all=pd.read_csv(f'{PATH}/train.csv',dtype=train_types,parse_dates=['date'],infer_datetime_format=True)

CPU times: user 1min 13s, sys: 20.9 s, total: 1min 34s
Wall time: 1min 32s


In [32]:
%time df_item=pd.read_csv(f'{PATH}/items.csv').set_index('item_nbr')

CPU times: user 7.67 ms, sys: 13.5 ms, total: 21.2 ms
Wall time: 23 ms


In [33]:
%time df_stores=pd.read_csv(f'{PATH}/stores.csv').set_index('store_nbr')

CPU times: user 7.96 ms, sys: 1.46 ms, total: 9.43 ms
Wall time: 9.42 ms


In [17]:
%time df_all=pd.merge(df_all,df_item,how='left',on=['item_nbr'])

CPU times: user 32.3 s, sys: 15.5 s, total: 47.8 s
Wall time: 50.1 s


In [18]:
%time df_all=pd.merge(df_all,df_stores,how='left',on=['store_nbr'])

CPU times: user 32.3 s, sys: 26.6 s, total: 58.9 s
Wall time: 1min 2s


In [30]:
%time df_test=pd.read_csv(f'{PATH}/test.csv',dtype=train_types,parse_dates=['date'],infer_datetime_format=True)

CPU times: user 1.81 s, sys: 276 ms, total: 2.09 s
Wall time: 2 s


In [34]:
%time df_test=pd.merge(df_test,df_item,how='left',on=['item_nbr'])

CPU times: user 843 ms, sys: 102 ms, total: 945 ms
Wall time: 730 ms


In [35]:
%time df_test=pd.merge(df_test,df_stores,how='left',on=['store_nbr'])

CPU times: user 983 ms, sys: 150 ms, total: 1.13 s
Wall time: 827 ms


In [ ]:
#n = 125497040 #number of records in file (excludes header)
#s = 500000 #desired sample size
#%time select = sorted(random.sample(range(1,n+1),s))
#skip  = tuple(set(range(1,n+1)) - set(select))
#%time df_train = pd.read_csv(f'{PATH}/train.csv',skiprows=skip,parse_dates=["date"],dtype=types)

In [19]:
df_all.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster
0,0,2013-01-01,25,103665,7.0,NaN,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,1.0,NaN,GROCERY I,1045,0,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,2.0,NaN,GROCERY I,1045,0,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,1.0,NaN,GROCERY I,1030,0,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,1.0,NaN,DELI,2644,1,Salinas,Santa Elena,D,1


In [20]:
df_all.isnull().sum()*100/len(df_all)

id              0.000000
date            0.000000
store_nbr       0.000000
item_nbr        0.000000
unit_sales      0.000000
onpromotion    17.257499
family          0.000000
class           0.000000
perishable      0.000000
city            0.000000
state           0.000000
type            0.000000
cluster         0.000000
dtype: float64

In [21]:
df_all.dtypes

id                      int64
date           datetime64[ns]
store_nbr                int8
item_nbr                int64
unit_sales            float64
onpromotion            object
family                 object
class                   int64
perishable              int64
city                   object
state                  object
type                   object
cluster                 int64
dtype: object

In [22]:
df_all.onpromotion.fillna(False,inplace=True)
df_all.onpromotion=df_all.onpromotion.map({'False':False,'True':True})
df_all.onpromotion=df_all.onpromotion.astype(bool)

In [23]:
df_all.dtypes

id                      int64
date           datetime64[ns]
store_nbr                int8
item_nbr                int64
unit_sales            float64
onpromotion              bool
family                 object
class                   int64
perishable              int64
city                   object
state                  object
type                   object
cluster                 int64
dtype: object

In [24]:
%time df_all.describe(include='all')

CPU times: user 1min 11s, sys: 18.5 s, total: 1min 30s
Wall time: 1min 28s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040,125497040,1.254970e+08,1.254970e+08,125497040,125497040,125497040,1.254970e+08
unique,NaN,1684,NaN,NaN,NaN,2,33,NaN,NaN,22,16,5,NaN
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False,GROCERY I,NaN,NaN,Quito,Pichincha,D,NaN
freq,NaN,118194,NaN,NaN,NaN,96028767,45628065,NaN,NaN,51982641,54550923,46801370,NaN
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,8.554865e+00,NaN,NaN,1.964087e+03,2.526158e-01,NaN,NaN,NaN,8.727105e+00
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.360515e+01,NaN,NaN,1.151258e+03,4.345124e-01,NaN,NaN,NaN,4.626750e+00
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN,NaN,1.002000e+03,0.000000e+00,NaN,NaN,NaN,1.000000e+00
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN,NaN,1.050000e+03,0.000000e+00,NaN,NaN,NaN,4.000000e+00


In [25]:
for col in df_all:
    print(col ,len(df_all[col].unique()))

id 125497040
date 1684
store_nbr 54
item_nbr 4036
unit_sales 258474
onpromotion 2
family 33
class 334
perishable 2
city 22
state 16
type 5
cluster 17


In [8]:
PATH2="/Users/franciumpnc/Documents/ML/AQM/fastai/courses/ml1/tmp"
#df_all.to_feather(f'{PATH2}/Grocery_train_items.csv')

In [9]:
!ls {PATH2}

Grocery_train_items.csv keep_cols.npy           titanic-raw
bulldozers-raw          subs_cols.npy           titanic-test-raw


In [10]:
%time df_all=pd.read_feather(f'{PATH2}/Grocery_train_items.csv')

CPU times: user 20 s, sys: 19.5 s, total: 39.5 s
Wall time: 48.3 s


In [11]:
df_all.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
       'family', 'class', 'perishable', 'city', 'state', 'type', 'cluster'],
      dtype='object')

In [12]:
#train_cats(df_all)

In [13]:
df_all.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster
0,0,2013-01-01,25,103665,7.0,True,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,1.0,True,GROCERY I,1045,0,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,2.0,True,GROCERY I,1045,0,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,1.0,True,GROCERY I,1030,0,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,1.0,True,DELI,2644,1,Salinas,Santa Elena,D,1


In [14]:
df_all.unit_sales=np.log1p(np.clip(df_all.unit_sales,0,None))

In [15]:
df_all.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster
0,0,2013-01-01,25,103665,2.079442,True,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,0.693147,True,GROCERY I,1045,0,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,1.098612,True,GROCERY I,1045,0,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,0.693147,True,GROCERY I,1030,0,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,0.693147,True,DELI,2644,1,Salinas,Santa Elena,D,1


In [16]:
%time add_datepart(df_all,'date')

CPU times: user 1min 52s, sys: 1min 27s, total: 3min 19s
Wall time: 3min 14s


In [37]:
%time add_datepart(df_test,'date')

CPU times: user 5.63 s, sys: 685 ms, total: 6.32 s
Wall time: 3.88 s


In [17]:
#??add_datepart

In [18]:
df_all.head()

,id,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25,103665,2.079442,True,BREAD/BAKERY,2712,1,Salinas,Santa Elena,...,1,1,1,False,True,False,True,False,True,1356998400
1,1,25,105574,0.693147,True,GROCERY I,1045,0,Salinas,Santa Elena,...,1,1,1,False,True,False,True,False,True,1356998400
2,2,25,105575,1.098612,True,GROCERY I,1045,0,Salinas,Santa Elena,...,1,1,1,False,True,False,True,False,True,1356998400
3,3,25,108079,0.693147,True,GROCERY I,1030,0,Salinas,Santa Elena,...,1,1,1,False,True,False,True,False,True,1356998400
4,4,25,108701,0.693147,True,DELI,2644,1,Salinas,Santa Elena,...,1,1,1,False,True,False,True,False,True,1356998400


In [19]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()
#??proc_df

In [20]:
%time n_valid = 3370465 # 
%time n_trn = len(df_all)-n_valid
%time train, valid = split_vals(df_all, n_trn)
%time train.shape, valid.shape

CPU times: user 11 µs, sys: 1 µs, total: 12 µs
Wall time: 16.9 µs
CPU times: user 19 µs, sys: 1e+03 ns, total: 20 µs
Wall time: 24.1 µs
CPU times: user 19 s, sys: 32.1 s, total: 51.1 s
Wall time: 47.3 s
CPU times: user 18 µs, sys: 9 µs, total: 27 µs
Wall time: 31 µs


((122126575, 25), (3370465, 25))

In [21]:
%time nas={}
#na_dict={onpromotion:na_onpromotion}
%time trn,y,nas=proc_df(train,y_fld='unit_sales',na_dict=nas)
%time val,y_val,nas=proc_df(valid,y_fld='unit_sales',na_dict=nas)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
CPU times: user 1min 59s, sys: 3min 22s, total: 5min 21s
Wall time: 6min
CPU times: user 4.01 s, sys: 1.79 s, total: 5.8 s
Wall time: 3.93 s


In [22]:
trn.head()

,id,store_nbr,item_nbr,onpromotion,family,class,perishable,city,state,type,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25,103665,True,6,2712,1,21,14,4,...,1,1,1,False,True,False,True,False,True,1356998400
1,1,25,105574,True,13,1045,0,21,14,4,...,1,1,1,False,True,False,True,False,True,1356998400
2,2,25,105575,True,13,1045,0,21,14,4,...,1,1,1,False,True,False,True,False,True,1356998400
3,3,25,108079,True,13,1030,0,21,14,4,...,1,1,1,False,True,False,True,False,True,1356998400
4,4,25,108701,True,10,2644,1,21,14,4,...,1,1,1,False,True,False,True,False,True,1356998400


In [23]:
#trn.plot(column='onpromotion')
trn['onpromotion'].value_counts(normalize=True) * 100

False    76.166664
True     23.833336
Name: onpromotion, dtype: float64

In [24]:
set_rf_samples(1_000_000)

In [ ]:
#??set_rf_samples

In [25]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
                m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [26]:
%time x=np.array(trn,dtype=np.float32)

CPU times: user 4min 3s, sys: 5min 11s, total: 9min 15s
Wall time: 11min 36s


In [27]:
m=RandomForestRegressor(n_estimators=20,min_samples_leaf=3,oob_score=False,n_jobs=-1)
%time m.fit(x,y)
#m.score(X_valid,y_valid)
%time print_score(m)

CPU times: user 20min 39s, sys: 5min 16s, total: 25min 56s
Wall time: 9min 54s
[0.5617250546969699, 0.6012621067783255, 0.594284865472217, 0.5188489643311528]
CPU times: user 29min 58s, sys: 5min 20s, total: 35min 18s
Wall time: 12min 33s


In [ ]:
%time print_score(m)

In [ ]:
%time add_datepart(df_all,'date')

In [ ]:
print_score(m)

In [38]:
test,_,_=proc_df(df_test,y_fld=None)

In [39]:
test.head()

,id,store_nbr,item_nbr,onpromotion,family,class,perishable,city,state,type,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,125497040,1,96995,1,13,1093,0,19,13,4,...,16,2,228,False,False,False,False,False,False,1502841600
1,125497041,1,99197,1,13,1067,0,19,13,4,...,16,2,228,False,False,False,False,False,False,1502841600
2,125497042,1,103501,1,8,3008,0,19,13,4,...,16,2,228,False,False,False,False,False,False,1502841600
3,125497043,1,103520,1,13,1028,0,19,13,4,...,16,2,228,False,False,False,False,False,False,1502841600
4,125497044,1,103665,1,6,2712,1,19,13,4,...,16,2,228,False,False,False,False,False,False,1502841600


In [40]:
pred=m.predict(test)

In [41]:
ids=test.id.to_numpy()
A=pd.DataFrame()
A['id']=ids
A['unit_sales']=pred
A.to_csv('Groceryfile.csv',index=False)

In [43]:
!head Groceryfile.csv

id,unit_sales
125497040,1.3916092230326687
125497041,1.5486612387754548
125497042,1.334743808570055
125497043,1.3388180837246644
125497044,1.263197479910774
125497045,1.8993914522739996
125497046,2.4031797071945986
125497047,2.4031797071945986
125497048,1.1060677306446576
